In [1]:
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
from tqdm import tqdm
import time
import random
import os
from IPython.display import display, HTML
import csv
from tqdm.notebook import tqdm


# Notes

### Offensive vs Defensive Styles

1. Man-to-Man Defense vs. Pass-Heavy Offense:
Strengths: Man coverage can effectively neutralize receivers, forcing the quarterback to make precise throws. It can also disrupt timing-based passing attacks and limit big plays.
Weaknesses: If the pass rush is not effective, the defense may struggle to cover receivers for an extended period, leading to potential big gains for the offense.


2. Zone Defense vs. Spread Offense:
Strengths: Zone coverage can provide better overall field coverage, making it difficult for the offense to exploit specific matchups. It can also force the quarterback to make difficult reads.
Weaknesses: Against a spread offense with quick, precise passes, zone coverage may leave gaps in the defense, allowing the offense to find open receivers easily.


3. Blitzing Defense vs. Strong Pocket Passer:
Strengths: Blitzing can put significant pressure on the quarterback, forcing quicker throws and disrupting the passing game.
Weaknesses: A strong pocket passer may exploit the blitz by quickly identifying and targeting hot routes or by releasing the ball before the pressure arrives, potentially leading to big plays.


4. Conservative Defense vs. Run-Heavy Offense:
Strengths: A conservative defense can focus on stopping the run and limiting big plays. It can also maintain gap discipline, making it harder for the offense to gain significant yardage on the ground.
Weaknesses: Against a potent run-heavy offense, a conservative defense may struggle to consistently stop the run and may get worn down over time.


5. Play-Action Offense vs. Aggressive Defense:
Strengths: An offense that effectively uses play-action can catch aggressive defenders out of position, creating opportunities for big plays downfield.
Weaknesses: An aggressive defense that bites on play-action fakes can leave open passing lanes and vulnerable to deep passes.


6. Power Running Offense vs. Blitzing Defense:
Strengths: A power running offense can wear down a blitzing defense and exploit the gaps created by defenders leaving their positions.
Weaknesses: Blitzing defenders can penetrate the backfield quickly and disrupt the running game, leading to tackles for loss or minimal gains.


7. West Coast Offense vs. Man Coverage Defense:
Strengths: The West Coast offense's quick, short passing game can create separation from man coverage defenders and lead to yards after the catch.
Weaknesses: If the man coverage defenders are physical and jam receivers at the line of scrimmage, it can disrupt the timing of West Coast plays.

# Code

In [1]:
## Testing new 3 

import csv
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm.notebook import tqdm

# Load the data
head_coaches = pd.read_csv('./data/head_coaches.csv')

# Initialize the progress bar
progress_bar = tqdm(total=head_coaches.shape[0], desc='Processing URLs')

# Loop through the rows of the DataFrame
for idx, row in head_coaches.iterrows():
    url = row['url']
    name = row['name']

    try:
        # Make a request to the website
        r = requests.get(url)
        r_html = r.text

        # Use BeautifulSoup to parse the HTML content
        soup = BeautifulSoup(r_html, 'html.parser')

        # Find the table body
        table_body = soup.find('tbody')

        if table_body is not None:
            # Prepare a CSV file to write to
            # Replace spaces in the name with underscores and append .csv
            filename = name.replace(' ', '_') + '.csv'
            with open(f'./data/head_coaches_detailed/{filename}', 'w', newline='') as csvfile:
                fieldnames = ['year_id', 'team', 'league_id', 'wins', 'losses', 'ties', 'wins_playoffs', 'losses_playoffs']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

                writer.writeheader()

                # Loop over each row
                for row in table_body.find_all('tr'):
                    # Find each cell
                    year_id = row.find('th', attrs={'data-stat': 'year_id'}).text
                    team = row.find('td', attrs={'data-stat': 'team'}).text
                    league_id = row.find('td', attrs={'data-stat': 'league_id'}).text
                    wins = row.find('td', attrs={'data-stat': 'wins'}).text
                    losses = row.find('td', attrs={'data-stat': 'losses'}).text
                    ties = row.find('td', attrs={'data-stat': 'ties'}).text
                    wins_playoffs = row.find('td', attrs={'data-stat': 'wins_playoffs'}).text
                    losses_playoffs = row.find('td', attrs={'data-stat': 'losses_playoffs'}).text

                    # Write to the CSV file
                    writer.writerow({
                        'year_id': year_id,
                        'team': team,
                        'league_id': league_id,
                        'wins': wins,
                        'losses': losses,
                        'ties': ties,
                        'wins_playoffs': wins_playoffs if wins_playoffs else '0', # If no data present, write '0'
                        'losses_playoffs': losses_playoffs if losses_playoffs else '0', # If no data present, write '0'
                    })
    except (requests.exceptions.RequestException, AttributeError, ValueError) as e:
        print(f"Error occurred while processing '{name}': {str(e)}")

    # Update the progress bar
    progress_bar.update(1)

# Close the progress bar
progress_bar.close()


Processing URLs:   0%|          | 0/518 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
### LOOP

# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
# }

# Read the CSV file containing coach URLs
df_url = pd.read_csv('./data/head_coaches.csv')

# Loop through each row in the DataFrame
for index, row in df_url.iterrows():
    coach_name = row['name']
    coach_name_with_underscore = coach_name.replace(' ', '_')
    url = row['url']

    try:
#        response = requests.get(url, headers=headers)
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes

        soup = BeautifulSoup(response.text, "html.parser")

        # Find the div that contains the table
        div = soup.find("div", {"id": "all_coaching_history"})

        # The table is in a comment, so find the comment
        comment = div.find(string=lambda text: isinstance(text, Comment))

        # Parse the comment as HTML
        table = BeautifulSoup(comment, "html.parser").find("table", {"id": "coaching_history"})

        # Prepare list to store data
        data = []

        # Iterate through each row in the table (skipping the first header row)
        for row in table.find_all("tr")[1:]:
            year_id = row.find("th", {"data-stat": "year_id"}).text.strip()
            coach_age = row.find("td", {"data-stat": "coach_age"}).text.strip()

            # Check if team name is a link or not
            employer_data = row.find("td", {"data-stat": "coach_employer"})
            if employer_data.a is not None:
                coach_employer = employer_data.a.text.strip()
            else:
                coach_employer = employer_data.text.strip()

            coach_role = row.find("td", {"data-stat": "coach_role"}).text.strip()
            
            ##### ADDING MORE MANUALLY

            #<tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2016/">2016</a></th><td class="right " data-stat="age" >53</td><td class="left " data-stat="team" ><a href="/teams/gnb/2016.htm">GNB</a></td><td class="left " data-stat="league_id" >NFL</td><td class="right " data-stat="g" ><a href="/teams/gnb/2016_games.htm" title="Click for season gamelogs">16</a></td><td class="right " data-stat="wins" >10</td><td class="right " data-stat="losses" >6</td><td class="right iz" data-stat="ties" >0</td><td class="right " data-stat="win_loss_perc" >.625</td><td class="right " data-stat="srs_total" >2.8</td><td class="right " data-stat="srs_offense" >4.9</td><td class="right " data-stat="srs_defense" >-2.0</td><td class="right " data-stat="g_playoffs" >3</td><td class="right " data-stat="wins_playoffs" >2</td><td class="right " data-stat="losses_playoffs" >1</td><td class="right " data-stat="win_loss_playoffs_perc" >.667</td><td class="right " data-stat="rank_team" >1</td><td class="right " data-stat="chall_num" >5</td><td class="right " data-stat="chall_won" >3</td><td class="left " data-stat="coach_remarks" > </td></tr>
            wins = row.find("td", {"data-stat": "wins"}).text
            losses = row.find("td", {"data-stat": "losses"}).text
            ties = row.find("td", {"data-stat": "ties"}).text
            wins_playoffs = row.find("td", {"data-stat": "wins_playoffs"}).text
            losses_playoffs = row.find("td", {"data-stat": "losses_playoffs"}).text


            ##### ADDING MORE MANUALLY END

            data.append([year_id, coach_age, coach_employer, coach_role, wins, losses, ties, wins_playoffs, losses_playoffs])

        # Convert list to DataFrame
        df = pd.DataFrame(data, columns=["Year", "Age", "Employer", "Role", "Wins", "Losses", "Ties", "Wins_Playoffs", "Losses_Playoffs"])

        # Write DataFrame to CSV with the modified coach's name as the file name
        file_name = f"./data/head_coaches_detailed/{coach_name_with_underscore}.csv"
        df.to_csv(file_name, index=False)

        print(f"CSV file '{file_name}' saved successfully.")

        # Pause for a short duration to avoid overwhelming the server
        time.sleep(5)  # Adjust the duration (in seconds) as needed

    except (requests.exceptions.RequestException, AttributeError, ValueError) as e:
        print(f"Error occurred while processing '{coach_name}': {str(e)}")


In [2]:
## Loop testing

# Read the CSV file containing coach URLs
df_url = pd.read_csv('./data/head_coaches.csv')

# Loop through each row in the DataFrame
for index, row in df_url.iterrows():
    coach_name = row['name']
    coach_name_with_underscore = coach_name.replace(' ', '_')
    url = row['url']

    try:
#        response = requests.get(url, headers=headers)
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes

        soup = BeautifulSoup(response.text, "html.parser")


        # Find the table body
        table_body = soup.find('tbody')

        # Prepare a CSV file to write to
        with open(f'./data/head_coaches_detailed/{coach_name_with_underscore}', 'w', newline='') as csvfile:
            fieldnames = ['year_id', 'team', 'league_id', 'wins', 'losses', 'ties', 'wins_playoffs', 'losses_playoffs']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()

            # Loop over each row
            for row in table_body.find_all('tr'):
                # Find each cell
                year_id = row.find('th', attrs={'data-stat': 'year_id'}).text
                team = row.find('td', attrs={'data-stat': 'team'}).text
                league_id = row.find('td', attrs={'data-stat': 'league_id'}).text
                wins = row.find('td', attrs={'data-stat': 'wins'}).text
                losses = row.find('td', attrs={'data-stat': 'losses'}).text
                ties = row.find('td', attrs={'data-stat': 'ties'}).text
                wins_playoffs = row.find('td', attrs={'data-stat': 'wins_playoffs'}).text
                losses_playoffs = row.find('td', attrs={'data-stat': 'losses_playoffs'}).text

                # Write to the CSV file
                writer.writerow({
                    'year_id': year_id,
                    'team': team,
                    'league_id': league_id,
                    'wins': wins,
                    'losses': losses,
                    'ties': ties,
                    'wins_playoffs': wins_playoffs if wins_playoffs else '0', # If no data present, write '0'
                    'losses_playoffs': losses_playoffs if losses_playoffs else '0', # If no data present, write '0'
                })
                
                print(f"CSV file '{coach_name_with_underscore}' saved successfully.")

                # Pause for a short duration to avoid overwhelming the server
                time.sleep(5)  # Adjust the duration (in seconds) as needed
                
                
    except (requests.exceptions.RequestException, AttributeError, ValueError) as e:
        print(f"Error occurred while processing '{coach_name}': {str(e)}")

CSV file 'Don_Shula' saved successfully.
CSV file 'Don_Shula' saved successfully.
CSV file 'Don_Shula' saved successfully.
CSV file 'Don_Shula' saved successfully.
CSV file 'Don_Shula' saved successfully.
CSV file 'Don_Shula' saved successfully.
CSV file 'Don_Shula' saved successfully.
CSV file 'Don_Shula' saved successfully.
CSV file 'Don_Shula' saved successfully.
CSV file 'Don_Shula' saved successfully.
CSV file 'Don_Shula' saved successfully.
CSV file 'Don_Shula' saved successfully.
CSV file 'Don_Shula' saved successfully.
CSV file 'Don_Shula' saved successfully.


KeyboardInterrupt: 

In [ ]:
## Download each html file instead
## Alt method

# # Load the CSV file into a pandas DataFrame
# csv_file = './data/head_coaches.csv'  # Replace this with the path to your CSV file
# df = pd.read_csv(csv_file)

# # Create the 'output' folder if it doesn't exist
# os.makedirs('output', exist_ok=True)

# # Loop through the 'url' column and download HTML for each URL
# for url in df['url']:
#     try:
#         response = requests.get(url)
#         response.raise_for_status()  # Check for any errors

#         # Extract the filename from the URL
#         filename = os.path.join('output', url.split('/')[-1] + '.html')

#         # Save the HTML content to a file
#         with open(filename, 'wb') as f:
#             f.write(response.content)

#         print(f"Downloaded HTML from {url} and saved as {filename}")
        
#         # Introduce a delay to avoid rate-limiting
#         time.sleep(3)  # Wait for 5 seconds before the next request

#     except Exception as e:
#         print(f"Error downloading {url}: {e}")


In [3]:
# Load the data
head_coaches = pd.read_csv('./data/head_coaches.csv')

# Initialize the progress bar
progress_bar = tqdm(total=head_coaches.shape[0], desc='Processing URLs')

# Loop through the rows of the DataFrame
for idx, row in head_coaches.iterrows():
    url = row['url']
    name = row['name']

    # Make a request to the website
    r = requests.get(url)
    r_html = r.text

    # Use BeautifulSoup to parse the HTML content
    soup = BeautifulSoup(r_html, 'html.parser')

    # Find the table body
    table_body = soup.find('tbody')
        
    filename = name.replace(' ', '_') + '.csv'
    with open(f'./data/head_coaches_detailed/{filename}', 'w', newline='') as csvfile:
        
        ##### ENTERING TRY
        try:
            fieldnames = ['year_id', 'team', 'league_id', 'wins', 'losses', 'ties', 'wins_playoffs', 'losses_playoffs']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()

            # Loop over each row
            for row in table_body.find_all('tr'):
                # Find each cell
                year_id = row.find('th', attrs={'data-stat': 'year_id'}).text
                team = row.find('td', attrs={'data-stat': 'team'}).text
                league_id = row.find('td', attrs={'data-stat': 'league_id'}).text
                wins = row.find('td', attrs={'data-stat': 'wins'}).text
                losses = row.find('td', attrs={'data-stat': 'losses'}).text
                ties = row.find('td', attrs={'data-stat': 'ties'}).text
                wins_playoffs = row.find('td', attrs={'data-stat': 'wins_playoffs'}).text
                losses_playoffs = row.find('td', attrs={'data-stat': 'losses_playoffs'}).text

                # Write to the CSV file
                writer.writerow({
                    'year_id': year_id,
                    'team': team,
                    'league_id': league_id,
                    'wins': wins,
                    'losses': losses,
                    'ties': ties,
                    'wins_playoffs': wins_playoffs if wins_playoffs else '0', # If no data present, write '0'
                    'losses_playoffs': losses_playoffs if losses_playoffs else '0', # If no data present, write '0'
                })


                print(f"CSV file '{coach_name_with_underscore}' saved successfully.")

                # Pause for a short duration to avoid overwhelming the server
                time.sleep(5)  # Adjust the duration (in seconds) as needed
        
        ##### ENTERING EXCEPT
        except (requests.exceptions.RequestException, AttributeError, ValueError) as e:
            print(f"Error occurred while processing '{coach_name}': {str(e)}")
    
    # Update the progress bar
    progress_bar.update(1)

# Close the progress bar
progress_bar.close()

Processing URLs:   0%|          | 0/518 [00:00<?, ?it/s]

CSV file 'Don_Shula' saved successfully.
CSV file 'Don_Shula' saved successfully.


KeyboardInterrupt: 

In [4]:
import pandas as pd
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import csv
import time

# Load the data
head_coaches = pd.read_csv('./data/head_coaches.csv')

# Initialize the progress bar
progress_bar = tqdm(total=head_coaches.shape[0], desc='Processing URLs')

# Loop through the rows of the DataFrame
for idx, row in head_coaches.iterrows():
    url = row['url']
    name = row['name']

    # Make a request to the website
    r = requests.get(url)
    r_html = r.text

    # Use BeautifulSoup to parse the HTML content
    soup = BeautifulSoup(r_html, 'html.parser')

    # Find the table body
    table_body = soup.find('tbody')

    filename = name.replace(' ', '_') + '.csv'
    with open(f'./data/head_coaches_detailed/{filename}', 'w', newline='') as csvfile:
        fieldnames = ['year_id', 'team', 'league_id', 'wins', 'losses', 'ties', 'wins_playoffs', 'losses_playoffs']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()

        # Loop over each row
        for table_row in table_body.find_all('tr'):  # Change the variable name to table_row
            year_id = table_row.find('th', attrs={'data-stat': 'year_id'}).text
            team = table_row.find('td', attrs={'data-stat': 'team'}).text
            league_id = table_row.find('td', attrs={'data-stat': 'league_id'}).text
            wins = table_row.find('td', attrs={'data-stat': 'wins'}).text
            losses = table_row.find('td', attrs={'data-stat': 'losses'}).text
            ties = table_row.find('td', attrs={'data-stat': 'ties'}).text
            wins_playoffs = table_row.find('td', attrs={'data-stat': 'wins_playoffs'}).text
            losses_playoffs = table_row.find('td', attrs={'data-stat': 'losses_playoffs'}).text

            writer.writerow({
                'year_id': year_id,
                'team': team,
                'league_id': league_id,
                'wins': wins,
                'losses': losses,
                'ties': ties,
                'wins_playoffs': wins_playoffs if wins_playoffs else '0',
                'losses_playoffs': losses_playoffs if losses_playoffs else '0',
            })

            print(f"CSV file '{filename}' saved successfully.")

            # Pause for a short duration to avoid overwhelming the server
            time.sleep(5)  # Adjust the duration (in seconds) as needed

    # Update the progress bar
    progress_bar.update(1)

# Close the progress bar
progress_bar.close()



Processing URLs:   0%|                                  | 0/518 [00:00<?, ?it/s]

CSV file 'Don_Shula.csv' saved successfully.
CSV file 'Don_Shula.csv' saved successfully.
CSV file 'Don_Shula.csv' saved successfully.


KeyboardInterrupt: 

In [5]:
import pandas as pd
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import csv
import time

# Load the data
head_coaches = pd.read_csv('./data/head_coaches.csv')

# Initialize the progress bar
progress_bar = tqdm(total=head_coaches.shape[0], desc='Processing URLs')

# Loop through the rows of the DataFrame
for idx, row in head_coaches.iterrows():
    url = row['url']
    name = row['name']

    # Make a request to the website
    r = requests.get(url)
    r_html = r.text

    # Use BeautifulSoup to parse the HTML content
    soup = BeautifulSoup(r_html, 'html.parser')

    # Find the table body
    table_body = soup.find('tbody')
    rows_in_table = table_body.find_all('tr')

    filename = name.replace(' ', '_') + '.csv'
    with open(f'./data/head_coaches_detailed/{filename}', 'w', newline='') as csvfile:
        fieldnames = ['year_id', 'team', 'league_id', 'wins', 'losses', 'ties', 'wins_playoffs', 'losses_playoffs']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()

        # Update the progress bar description
        progress_bar.set_description(f'Processing {name}')

        # Loop over each row
        for table_row in rows_in_table:  # Change the variable name to table_row
            year_id = table_row.find('th', attrs={'data-stat': 'year_id'}).text
            team = table_row.find('td', attrs={'data-stat': 'team'}).text
            league_id = table_row.find('td', attrs={'data-stat': 'league_id'}).text
            wins = table_row.find('td', attrs={'data-stat': 'wins'}).text
            losses = table_row.find('td', attrs={'data-stat': 'losses'}).text
            ties = table_row.find('td', attrs={'data-stat': 'ties'}).text
            wins_playoffs = table_row.find('td', attrs={'data-stat': 'wins_playoffs'}).text
            losses_playoffs = table_row.find('td', attrs={'data-stat': 'losses_playoffs'}).text

            writer.writerow({
                'year_id': year_id,
                'team': team,
                'league_id': league_id,
                'wins': wins,
                'losses': losses,
                'ties': ties,
                'wins_playoffs': wins_playoffs if wins_playoffs else '0',
                'losses_playoffs': losses_playoffs if losses_playoffs else '0',
            })

            print(f"CSV file '{filename}' saved successfully.")

            # Pause for a short duration to avoid overwhelming the server
            time.sleep(5)  # Adjust the duration (in seconds) as needed

    # Update the progress bar for the number of rows processed for this coach
    progress_bar.update(len(rows_in_table))

# Close the progress bar
progress_bar.close()




Processing URLs:   0%|                                  | 0/518 [00:50<?, ?it/s]


Processing Don Shula:   0%|                             | 0/518 [00:00<?, ?it/s]

CSV file 'Don_Shula.csv' saved successfully.
CSV file 'Don_Shula.csv' saved successfully.
CSV file 'Don_Shula.csv' saved successfully.
CSV file 'Don_Shula.csv' saved successfully.
CSV file 'Don_Shula.csv' saved successfully.


KeyboardInterrupt: 

In [6]:
import pandas as pd
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import csv
import time

# Load the data
head_coaches = pd.read_csv('./data/head_coaches.csv')

# Initialize the progress bar
progress_bar = tqdm(total=head_coaches.shape[0], desc='Processing URLs')

# Loop through the rows of the DataFrame
for idx, row in head_coaches.iterrows():
    url = row['url']
    name = row['name']

    # Make a request to the website
    r = requests.get(url)
    r_html = r.text

    # Use BeautifulSoup to parse the HTML content
    soup = BeautifulSoup(r_html, 'html.parser')

    # Find the table body
    table_body = soup.find('tbody')
    rows_in_table = table_body.find_all('tr')

    filename = name.replace(' ', '_') + '.csv'
    with open(f'./data/head_coaches_detailed/{filename}', 'w', newline='') as csvfile:
        fieldnames = ['year_id', 'team', 'league_id', 'wins', 'losses', 'ties', 'wins_playoffs', 'losses_playoffs']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()

        # Update the progress bar description
        progress_bar.set_description(f'Processing {name}')

        # Loop over each row
        for table_row in rows_in_table:
            year_id = table_row.find('th', attrs={'data-stat': 'year_id'}).text
            team = table_row.find('td', attrs={'data-stat': 'team'}).text
            league_id = table_row.find('td', attrs={'data-stat': 'league_id'}).text
            wins = table_row.find('td', attrs={'data-stat': 'wins'}).text
            losses = table_row.find('td', attrs={'data-stat': 'losses'}).text
            ties = table_row.find('td', attrs={'data-stat': 'ties'}).text
            wins_playoffs = table_row.find('td', attrs={'data-stat': 'wins_playoffs'}).text
            losses_playoffs = table_row.find('td', attrs={'data-stat': 'losses_playoffs'}).text

            writer.writerow({
                'year_id': year_id,
                'team': team,
                'league_id': league_id,
                'wins': wins,
                'losses': losses,
                'ties': ties,
                'wins_playoffs': wins_playoffs if wins_playoffs else '0',
                'losses_playoffs': losses_playoffs if losses_playoffs else '0',
            })

            print(f"CSV file '{filename}' saved successfully.")

            # Pause for a short duration to avoid overwhelming the server
            time.sleep(5)  # Adjust the duration (in seconds) as needed

    # Update the progress bar for the coach being processed
    progress_bar.update(1)

# Close the progress bar
progress_bar.close()



Processing Don Shula:   0%|                             | 0/518 [01:15<?, ?it/s]

Processing Don Shula:   0%|                             | 0/518 [00:00<?, ?it/s]

CSV file 'Don_Shula.csv' saved successfully.
CSV file 'Don_Shula.csv' saved successfully.


KeyboardInterrupt: 

In [ ]:
import csv
from bs4 import BeautifulSoup
import requests
import os
import time

# Read the input CSV file
with open('./data/head_coaches.csv', 'r') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        # Get the URL and coach's name
        url = row['url']
        name = row['name']

        # Make a request to the website
        r = requests.get(url)
        r_html = r.text

        # Use BeautifulSoup to parse the HTML content
        soup = BeautifulSoup(r_html, 'html.parser')

        # Find the table body
        table_body = soup.find('tbody')

        # Prepare an output CSV file with the coach's name
        filename = './data/head_coaches_detailed/' + name.replace(' ', '_') + '.csv'
        with open(filename, 'w', newline='') as csvfile:
            fieldnames = ['year_id', 'team', 'league_id', 'wins', 'losses', 'ties', 'wins_playoffs', 'losses_playoffs']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()

            # Loop over each row
            for row in table_body.find_all('tr'):
                # Find each cell
                year_id = row.find('th', attrs={'data-stat': 'year_id'}).text
                team = row.find('td', attrs={'data-stat': 'team'}).text
                league_id = row.find('td', attrs={'data-stat': 'league_id'}).text
                wins = row.find('td', attrs={'data-stat': 'wins'}).text
                losses = row.find('td', attrs={'data-stat': 'losses'}).text
                ties = row.find('td', attrs={'data-stat': 'ties'}).text
                wins_playoffs = row.find('td', attrs={'data-stat': 'wins_playoffs'}).text
                losses_playoffs = row.find('td', attrs={'data-stat': 'losses_playoffs'}).text

                # Write to the CSV file
                writer.writerow({
                    'year_id': year_id,
                    'team': team,
                    'league_id': league_id,
                    'wins': wins,
                    'losses': losses,
                    'ties': ties,
                    'wins_playoffs': wins_playoffs if wins_playoffs else '0', # If no data present, write '0'
                    'losses_playoffs': losses_playoffs if losses_playoffs else '0', # If no data present, write '0'
                })
                
                time.sleep(5)


In [2]:
import csv
from bs4 import BeautifulSoup
import requests
import os
from tqdm.notebook import tqdm
import time

# Read the input CSV file
with open('./data/head_coaches.csv', 'r') as csv_file:
    reader = list(csv.DictReader(csv_file))
    for row in tqdm(reader, desc='Processing URLs'):
        # Get the URL and coach's name
        url = row['url']
        name = row['name']

        # Make a request to the website
        r = requests.get(url)
        r_html = r.text

        # Use BeautifulSoup to parse the HTML content
        soup = BeautifulSoup(r_html, 'html.parser')

        # Find the table body
        table_body = soup.find('tbody')

        # Prepare an output CSV file with the coach's name
        filename = './data/head_coaches_detailed/' + name.replace(' ', '_') + '.csv'
        with open(filename, 'w', newline='') as csvfile:
            fieldnames = ['year_id', 'team', 'league_id', 'wins', 'losses', 'ties', 'wins_playoffs', 'losses_playoffs']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()

            # Loop over each row
            for row in table_body.find_all('tr'):
                # Find each cell
                year_id = row.find('th', attrs={'data-stat': 'year_id'}).text
                team = row.find('td', attrs={'data-stat': 'team'}).text
                league_id = row.find('td', attrs={'data-stat': 'league_id'}).text
                wins = row.find('td', attrs={'data-stat': 'wins'}).text
                losses = row.find('td', attrs={'data-stat': 'losses'}).text
                ties = row.find('td', attrs={'data-stat': 'ties'}).text
                wins_playoffs = row.find('td', attrs={'data-stat': 'wins_playoffs'}).text
                losses_playoffs = row.find('td', attrs={'data-stat': 'losses_playoffs'}).text

                # Write to the CSV file
                writer.writerow({
                    'year_id': year_id,
                    'team': team,
                    'league_id': league_id,
                    'wins': wins,
                    'losses': losses,
                    'ties': ties,
                    'wins_playoffs': wins_playoffs if wins_playoffs else '0', # If no data present, write '0'
                    'losses_playoffs': losses_playoffs if losses_playoffs else '0', # If no data present, write '0'
                })
                
                time.sleep(5)


In [1]:
#### WORKING 

import csv
from bs4 import BeautifulSoup
import requests
import os
from tqdm.notebook import tqdm
import time

# Read the input CSV file
with open('./data/head_coaches.csv', 'r') as csv_file:
    reader = list(csv.DictReader(csv_file))
    for row in tqdm(reader, desc='Processing URLs'):
        # Get the URL and coach's name
        url = row['url']
        name = row['name']

        try:
            # Make a request to the website
            r = requests.get(url)
            r_html = r.text

            # Use BeautifulSoup to parse the HTML content
            soup = BeautifulSoup(r_html, 'html.parser')

            # Find the table body
            table_body = soup.find('tbody')

            # Prepare an output CSV file with the coach's name
            filename = './data/head_coaches_detailed/' + name.replace(' ', '_') + '.csv'
            with open(filename, 'w', newline='') as csvfile:
                fieldnames = ['year_id', 'team', 'league_id', 'wins', 'losses', 'ties', 'wins_playoffs', 'losses_playoffs']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

                writer.writeheader()

                # Loop over each row
                for row in table_body.find_all('tr'):
                    # Find each cell
                    year_id = row.find('th', attrs={'data-stat': 'year_id'}).text
                    team = row.find('td', attrs={'data-stat': 'team'}).text
                    league_id = row.find('td', attrs={'data-stat': 'league_id'}).text
                    wins = row.find('td', attrs={'data-stat': 'wins'}).text
                    losses = row.find('td', attrs={'data-stat': 'losses'}).text
                    ties = row.find('td', attrs={'data-stat': 'ties'}).text
                    wins_playoffs = row.find('td', attrs={'data-stat': 'wins_playoffs'}).text
                    losses_playoffs = row.find('td', attrs={'data-stat': 'losses_playoffs'}).text

                    # Write to the CSV file
                    writer.writerow({
                        'year_id': year_id,
                        'team': team,
                        'league_id': league_id,
                        'wins': wins,
                        'losses': losses,
                        'ties': ties,
                        'wins_playoffs': wins_playoffs if wins_playoffs else '0', # If no data present, write '0'
                        'losses_playoffs': losses_playoffs if losses_playoffs else '0', # If no data present, write '0'
                    })
            
            time.sleep(5)

        except (requests.exceptions.RequestException, AttributeError, ValueError) as e:
            print(f"Error occurred while processing {name} with URL {url}: {str(e)}")


Processing URLs:   0%|          | 0/518 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [2]:
import csv
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

# Read the CSV file with pandas
df = pd.read_csv('./data/head_coaches.csv')

# Loop over each row in the dataframe
for index, row in df.iterrows():
    url = row['url']
    output_file_name = './data/head_coaches_detailed/' + row['name'] + '.csv'

    # Make a request to the website
    r = requests.get(url)
    r_html = r.text

    # Use BeautifulSoup to parse the HTML content
    soup = BeautifulSoup(r_html, 'html.parser')

    # Find the table body
    table_body = soup.find('tbody')

    # Prepare a CSV file to write to
    with open(output_file_name, 'w', newline='') as csvfile:
        fieldnames = ['year_id', 'team', 'league_id', 'wins', 'losses', 'ties', 'wins_playoffs', 'losses_playoffs']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()

        # Loop over each row
        for row in table_body.find_all('tr'):
            # Find each cell
            year_id = row.find('th', attrs={'data-stat': 'year_id'}).text
            team = row.find('td', attrs={'data-stat': 'team'}).text
            league_id = row.find('td', attrs={'data-stat': 'league_id'}).text
            wins = row.find('td', attrs={'data-stat': 'wins'}).text
            losses = row.find('td', attrs={'data-stat': 'losses'}).text
            ties = row.find('td', attrs={'data-stat': 'ties'}).text
            wins_playoffs = row.find('td', attrs={'data-stat': 'wins_playoffs'}).text
            losses_playoffs = row.find('td', attrs={'data-stat': 'losses_playoffs'}).text

            # Write to the CSV file
            writer.writerow({
                'year_id': year_id,
                'team': team,
                'league_id': league_id,
                'wins': wins,
                'losses': losses,
                'ties': ties,
                'wins_playoffs': wins_playoffs if wins_playoffs else '0', # If no data present, write '0'
                'losses_playoffs': losses_playoffs if losses_playoffs else '0', # If no data present, write '0'
            })

    # Sleep for 5 seconds to prevent overloading the server with requests
    time.sleep(5)


In [ ]:
# <div id="all_coaching_ranks" class="table_wrapper setup_commented commented">

# <div class="section_heading assoc_coaching_ranks" id="coaching_ranks_sh">
#   <span class="section_anchor" id="coaching_ranks_link" data-label="Team's Ranks"></span><h2>Team's Ranks</h2>    <div class="section_heading_text">
#       <ul><li>Team ranks, totals at bottom are percentiles</li>
#       </ul>
#     </div>
    		
# </div><div class="placeholder"></div>
# <!--

# <div class="table_container" id="div_coaching_ranks">
    
#     <table class="sortable stats_table" id="coaching_ranks" data-cols-to-freeze="1,2">
#     <caption>Team's Ranks Table</caption>
    

#    <colgroup><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col></colgroup>
#    <thead>

      
#       <tr class="over_header">
#          <th aria-label="" data-stat="" colspan="4" class=" over_header center" ></th>
#          <th aria-label="" data-stat="header_rall" colspan="4" class=" over_header center" >Overall</th>
#          <th aria-label="" data-stat="header_off" colspan="3" class=" over_header center" >Offense</th>
#          <th aria-label="" data-stat="header_off_rush" colspan="5" class=" over_header center" >Rushing Off</th>
#          <th aria-label="" data-stat="header_off_pass" colspan="5" class=" over_header center" >Passing Off</th>
#          <th aria-label="" data-stat="header_def" colspan="3" class=" over_header center" >Defense</th>
#          <th aria-label="" data-stat="header_def_rush" colspan="5" class=" over_header center" >Rushing Def</th>
#          <th aria-label="" data-stat="header_def_pass" colspan="5" class=" over_header center" >Passing Def</th>
#       </tr>
            
#       <tr>
#          <th aria-label="Year" data-stat="year_id" scope="col" class=" poptip sort_default_asc left" >Year</th>
#          <th aria-label="Tm" data-stat="team" scope="col" class=" poptip sort_default_asc show_partial_when_sorting left" >Tm</th>
#          <th aria-label="Role" data-stat="coordinator_type" scope="col" class=" poptip sort_default_asc center" data-tip="Coordinator type for this coach (OC = Offensive Coordinator, DC = Defensive Coordinator)" >Role</th>
#          <th aria-label="Tms" data-stat="teams_in_league" scope="col" class=" poptip sort_default_asc center" data-tip="Number of teams in the league this season" >Tms</th>
#          <th aria-label="WL%" data-stat="rank_win_percentage" scope="col" class=" poptip center" data-over-header="Overall" >WL%</th>
#          <th aria-label="T/G" data-stat="rank_takeaway_giveaway" scope="col" class=" poptip sort_default_asc center" data-tip="Team's Rank in Takeaway/Giveaway Ratio" data-over-header="Overall" >T/G</th>
#          <th aria-label="Pts&#177;" data-stat="rank_points_diff" scope="col" class=" poptip sort_default_asc center" data-tip="Team's Rank in Point Differential<br>(the lower the number,<br> the more they outscored their opponents)" data-over-header="Overall" >Pts&#177;</th>
#          <th aria-label="Yds&#177;" data-stat="rank_yds_diff" scope="col" class=" poptip sort_default_asc center" data-tip="Team's Rank in Yardage Differential<br>(the lower the number,<br> the more they outgained their opponents)" data-over-header="Overall" >Yds&#177;</th>
#          <th aria-label="Yds" data-stat="rank_off_yds" scope="col" class=" poptip sort_default_asc center" data-over-header="Offense" >Yds</th>
#          <th aria-label="Pts" data-stat="rank_off_pts" scope="col" class=" poptip sort_default_asc center" data-over-header="Offense" >Pts</th>
#          <th aria-label="GvA" data-stat="rank_off_turnovers" scope="col" class=" poptip sort_default_asc center" data-tip="Rank in giveaways<br>Interceptions + Fumbles Lost<br>Lower is better" data-over-header="Offense" >GvA</th>
#          <th aria-label="Att" data-stat="rank_off_rush_att" scope="col" class=" poptip sort_default_asc center" data-over-header="Rushing Off" >Att</th>
#          <th aria-label="Yds" data-stat="rank_off_rush_yds" scope="col" class=" poptip sort_default_asc center" data-over-header="Rushing Off" >Yds</th>
#          <th aria-label="TD" data-stat="rank_off_rush_td" scope="col" class=" poptip sort_default_asc center" data-over-header="Rushing Off" >TD</th>
#          <th aria-label="Y/A" data-stat="rank_off_rush_yds_per_att" scope="col" class=" poptip sort_default_asc center" data-over-header="Rushing Off" >Y/A</th>
#          <th aria-label="FL" data-stat="rank_off_fumbles_lost" scope="col" class=" poptip sort_default_asc center" data-over-header="Rushing Off" >FL</th>
#          <th aria-label="Att" data-stat="rank_off_pass_att" scope="col" class=" poptip sort_default_asc center" data-over-header="Passing Off" >Att</th>
#          <th aria-label="Yds" data-stat="rank_off_pass_yds" scope="col" class=" poptip sort_default_asc center" data-over-header="Passing Off" >Yds</th>
#          <th aria-label="TD" data-stat="rank_off_pass_td" scope="col" class=" poptip sort_default_asc center" data-over-header="Passing Off" >TD</th>
#          <th aria-label="Int" data-stat="rank_off_pass_int" scope="col" class=" poptip sort_default_asc center" data-over-header="Passing Off" >Int</th>
#          <th aria-label="NY/A" data-stat="rank_off_pass_net_yds_per_att" scope="col" class=" poptip sort_default_asc center" data-tip="Rank in Net Yards gained per pass attempt<br>(Passing Yards - Sack Yards) / (Passes Attempted + Times Sacked)" data-over-header="Passing Off" >NY/A</th>
#          <th aria-label="Yds" data-stat="rank_def_yds" scope="col" class=" poptip sort_default_asc center" data-over-header="Defense" >Yds</th>
#          <th aria-label="Pts" data-stat="rank_def_pts" scope="col" class=" poptip sort_default_asc center" data-over-header="Defense" >Pts</th>
#          <th aria-label="TkA" data-stat="rank_def_turnovers" scope="col" class=" poptip sort_default_asc center" data-tip="Rank in takeaways<br>Passes Interceptions + opponents Fumbles Lost<br>Lower is better" data-over-header="Defense" >TkA</th>
#          <th aria-label="Att" data-stat="rank_def_rush_att" scope="col" class=" poptip sort_default_asc center" data-over-header="Rushing Def" >Att</th>
#          <th aria-label="Yds" data-stat="rank_def_rush_yds" scope="col" class=" poptip sort_default_asc center" data-over-header="Rushing Def" >Yds</th>
#          <th aria-label="TD" data-stat="rank_def_rush_td" scope="col" class=" poptip sort_default_asc center" data-over-header="Rushing Def" >TD</th>
#          <th aria-label="Y/A" data-stat="rank_def_rush_yds_per_att" scope="col" class=" poptip sort_default_asc center" data-over-header="Rushing Def" >Y/A</th>
#          <th aria-label="FR" data-stat="rank_def_fumbles_rec" scope="col" class=" poptip sort_default_asc center" data-over-header="Rushing Def" >FR</th>
#          <th aria-label="Att" data-stat="rank_def_pass_att" scope="col" class=" poptip sort_default_asc center" data-over-header="Passing Def" >Att</th>
#          <th aria-label="Yds" data-stat="rank_def_pass_yds" scope="col" class=" poptip sort_default_asc center" data-over-header="Passing Def" >Yds</th>
#          <th aria-label="TD" data-stat="rank_def_pass_td" scope="col" class=" poptip sort_default_asc center" data-over-header="Passing Def" >TD</th>
#          <th aria-label="Int" data-stat="rank_def_pass_int" scope="col" class=" poptip sort_default_asc center" data-over-header="Passing Def" >Int</th>
#          <th aria-label="nY/A" data-stat="rank_def_pass_net_yds_per_att" scope="col" class=" poptip sort_default_asc center" data-tip="Rank in Net Yards gained per pass attempt by opposition<br>(Passing Yards - Sack Yards) / (Passes Attempted + Times Sacked)" data-over-header="Passing Def" >nY/A</th>
#       </tr>
#       </thead>
# <tbody><tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2000/">2000</a></th><td class="left " data-stat="team" ><a href="/teams/nor/2000.htm" title="New Orleans Saints">NOR</a></td><td class="left " data-stat="coordinator_type" >OC</td><td class="right " data-stat="teams_in_league" >31</td><td class="right " data-stat="rank_win_percentage" >9</td><td class="right " data-stat="rank_takeaway_giveaway" >8</td><td class="right " data-stat="rank_points_diff" >12</td><td class="right " data-stat="rank_yds_diff" >10</td><td class="right " data-stat="rank_off_yds" >10</td><td class="right " data-stat="rank_off_pts" >10</td><td class="right " data-stat="rank_off_turnovers" >9</td><td class="right " data-stat="rank_off_rush_att" >7</td><td class="right " data-stat="rank_off_rush_yds" >8</td><td class="right " data-stat="rank_off_rush_td" >12</td><td class="right " data-stat="rank_off_rush_yds_per_att" >16</td><td class="right " data-stat="rank_off_fumbles_lost" >8</td><td class="right " data-stat="rank_off_pass_att" >22</td><td class="right " data-stat="rank_off_pass_yds" >14</td><td class="right " data-stat="rank_off_pass_td" >9</td><td class="right " data-stat="rank_off_pass_int" >9</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >9</td><td class="right " data-stat="rank_def_yds" >8</td><td class="right " data-stat="rank_def_pts" >10</td><td class="right " data-stat="rank_def_turnovers" >8</td><td class="right " data-stat="rank_def_rush_att" >7</td><td class="right " data-stat="rank_def_rush_yds" >10</td><td class="right " data-stat="rank_def_rush_td" >20</td><td class="right " data-stat="rank_def_rush_yds_per_att" >20</td><td class="right " data-stat="rank_def_fumbles_rec" >10</td><td class="right " data-stat="rank_def_pass_att" >8</td><td class="right " data-stat="rank_def_pass_yds" >10</td><td class="right " data-stat="rank_def_pass_td" >9</td><td class="right " data-stat="rank_def_pass_int" >9</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >11</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2001/">2001</a></th><td class="left " data-stat="team" ><a href="/teams/nor/2001.htm" title="New Orleans Saints">NOR</a></td><td class="left " data-stat="coordinator_type" >OC</td><td class="right " data-stat="teams_in_league" >31</td><td class="right " data-stat="rank_win_percentage" >16</td><td class="right " data-stat="rank_takeaway_giveaway" >21</td><td class="right " data-stat="rank_points_diff" >24</td><td class="right " data-stat="rank_yds_diff" >13</td><td class="right " data-stat="rank_off_yds" >10</td><td class="right " data-stat="rank_off_pts" >13</td><td class="right " data-stat="rank_off_turnovers" >23</td><td class="right " data-stat="rank_off_rush_att" >20</td><td class="right " data-stat="rank_off_rush_yds" >18</td><td class="right " data-stat="rank_off_rush_td" >26</td><td class="right " data-stat="rank_off_rush_yds_per_att" >15</td><td class="right " data-stat="rank_off_fumbles_lost" >16</td><td class="right " data-stat="rank_off_pass_att" >6</td><td class="right " data-stat="rank_off_pass_yds" >10</td><td class="right " data-stat="rank_off_pass_td" >4</td><td class="right " data-stat="rank_off_pass_int" >24</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >16</td><td class="right " data-stat="rank_def_yds" >16</td><td class="right " data-stat="rank_def_pts" >27</td><td class="right " data-stat="rank_def_turnovers" >14</td><td class="right " data-stat="rank_def_rush_att" >15</td><td class="right " data-stat="rank_def_rush_yds" >14</td><td class="right " data-stat="rank_def_rush_td" >22</td><td class="right " data-stat="rank_def_rush_yds_per_att" >13</td><td class="right " data-stat="rank_def_fumbles_rec" >6</td><td class="right " data-stat="rank_def_pass_att" >1</td><td class="right " data-stat="rank_def_pass_yds" >17</td><td class="right " data-stat="rank_def_pass_td" >29</td><td class="right " data-stat="rank_def_pass_int" >20</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >28</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2002/">2002</a></th><td class="left " data-stat="team" ><a href="/teams/nor/2002.htm" title="New Orleans Saints">NOR</a></td><td class="left " data-stat="coordinator_type" >OC</td><td class="right " data-stat="teams_in_league" >32</td><td class="right " data-stat="rank_win_percentage" >11</td><td class="right " data-stat="rank_takeaway_giveaway" >9</td><td class="right " data-stat="rank_points_diff" >10</td><td class="right " data-stat="rank_yds_diff" >25</td><td class="right " data-stat="rank_off_yds" >19</td><td class="right " data-stat="rank_off_pts" >3</td><td class="right " data-stat="rank_off_turnovers" >17</td><td class="right " data-stat="rank_off_rush_att" >19</td><td class="right " data-stat="rank_off_rush_yds" >17</td><td class="right " data-stat="rank_off_rush_td" >11</td><td class="right " data-stat="rank_off_rush_yds_per_att" >16</td><td class="right " data-stat="rank_off_fumbles_lost" >22</td><td class="right " data-stat="rank_off_pass_att" >21</td><td class="right " data-stat="rank_off_pass_yds" >16</td><td class="right " data-stat="rank_off_pass_td" >3</td><td class="right " data-stat="rank_off_pass_int" >12</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >15</td><td class="right " data-stat="rank_def_yds" >27</td><td class="right " data-stat="rank_def_pts" >26</td><td class="right " data-stat="rank_def_turnovers" >3</td><td class="right " data-stat="rank_def_rush_att" >16</td><td class="right " data-stat="rank_def_rush_yds" >19</td><td class="right " data-stat="rank_def_rush_td" >27</td><td class="right " data-stat="rank_def_rush_yds_per_att" >26</td><td class="right " data-stat="rank_def_fumbles_rec" >4</td><td class="right " data-stat="rank_def_pass_att" >30</td><td class="right " data-stat="rank_def_pass_yds" >27</td><td class="right " data-stat="rank_def_pass_td" >25</td><td class="right " data-stat="rank_def_pass_int" >7</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >23</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2003/">2003</a></th><td class="left " data-stat="team" ><a href="/teams/nor/2003.htm" title="New Orleans Saints">NOR</a></td><td class="left " data-stat="coordinator_type" >OC</td><td class="right " data-stat="teams_in_league" >32</td><td class="right " data-stat="rank_win_percentage" >15</td><td class="right " data-stat="rank_takeaway_giveaway" >18</td><td class="right " data-stat="rank_points_diff" >17</td><td class="right " data-stat="rank_yds_diff" >15</td><td class="right " data-stat="rank_off_yds" >11</td><td class="right " data-stat="rank_off_pts" >14</td><td class="right " data-stat="rank_off_turnovers" >13</td><td class="right " data-stat="rank_off_rush_att" >15</td><td class="right " data-stat="rank_off_rush_yds" >11</td><td class="right " data-stat="rank_off_rush_td" >20</td><td class="right " data-stat="rank_off_rush_yds_per_att" >9</td><td class="right " data-stat="rank_off_fumbles_lost" >32</td><td class="right " data-stat="rank_off_pass_att" >8</td><td class="right " data-stat="rank_off_pass_yds" >8</td><td class="right " data-stat="rank_off_pass_td" >8</td><td class="right " data-stat="rank_off_pass_int" >1</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >14</td><td class="right " data-stat="rank_def_yds" >18</td><td class="right " data-stat="rank_def_pts" >14</td><td class="right " data-stat="rank_def_turnovers" >16</td><td class="right " data-stat="rank_def_rush_att" >25</td><td class="right " data-stat="rank_def_rush_yds" >27</td><td class="right " data-stat="rank_def_rush_td" >13</td><td class="right " data-stat="rank_def_rush_yds_per_att" >28</td><td class="right " data-stat="rank_def_fumbles_rec" >7</td><td class="right " data-stat="rank_def_pass_att" >7</td><td class="right " data-stat="rank_def_pass_yds" >8</td><td class="right " data-stat="rank_def_pass_td" >16</td><td class="right " data-stat="rank_def_pass_int" >20</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >15</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2004/">2004</a></th><td class="left " data-stat="team" ><a href="/teams/nor/2004.htm" title="New Orleans Saints">NOR</a></td><td class="left " data-stat="coordinator_type" >OC</td><td class="right " data-stat="teams_in_league" >32</td><td class="right " data-stat="rank_win_percentage" >14</td><td class="right " data-stat="rank_takeaway_giveaway" >10</td><td class="right " data-stat="rank_points_diff" >24</td><td class="right " data-stat="rank_yds_diff" >31</td><td class="right " data-stat="rank_off_yds" >15</td><td class="right " data-stat="rank_off_pts" >14</td><td class="right " data-stat="rank_off_turnovers" >12</td><td class="right " data-stat="rank_off_rush_att" >26</td><td class="right " data-stat="rank_off_rush_yds" >27</td><td class="right " data-stat="rank_off_rush_td" >8</td><td class="right " data-stat="rank_off_rush_yds_per_att" >21</td><td class="right " data-stat="rank_off_fumbles_lost" >8</td><td class="right " data-stat="rank_off_pass_att" >10</td><td class="right " data-stat="rank_off_pass_yds" >12</td><td class="right " data-stat="rank_off_pass_td" >16</td><td class="right " data-stat="rank_off_pass_int" >14</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >18</td><td class="right " data-stat="rank_def_yds" >32</td><td class="right " data-stat="rank_def_pts" >27</td><td class="right " data-stat="rank_def_turnovers" >8</td><td class="right " data-stat="rank_def_rush_att" >25</td><td class="right " data-stat="rank_def_rush_yds" >30</td><td class="right " data-stat="rank_def_rush_td" >23</td><td class="right " data-stat="rank_def_rush_yds_per_att" >31</td><td class="right " data-stat="rank_def_fumbles_rec" >1</td><td class="right " data-stat="rank_def_pass_att" >28</td><td class="right " data-stat="rank_def_pass_yds" >27</td><td class="right " data-stat="rank_def_pass_td" >19</td><td class="right " data-stat="rank_def_pass_int" >24</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >27</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2005/">2005</a></th><td class="left " data-stat="team" ><a href="/teams/sfo/2005.htm" title="San Francisco 49ers">SFO</a></td><td class="left " data-stat="coordinator_type" >OC</td><td class="right " data-stat="teams_in_league" >32</td><td class="right " data-stat="rank_win_percentage" >26</td><td class="right " data-stat="rank_takeaway_giveaway" >27</td><td class="right " data-stat="rank_points_diff" >32</td><td class="right " data-stat="rank_yds_diff" >32</td><td class="right " data-stat="rank_off_yds" >32</td><td class="right " data-stat="rank_off_pts" >30</td><td class="right " data-stat="rank_off_turnovers" >27</td><td class="right " data-stat="rank_off_rush_att" >20</td><td class="right " data-stat="rank_off_rush_yds" >17</td><td class="right " data-stat="rank_off_rush_td" >25</td><td class="right " data-stat="rank_off_rush_yds_per_att" >16</td><td class="right " data-stat="rank_off_fumbles_lost" >21</td><td class="right " data-stat="rank_off_pass_att" >31</td><td class="right " data-stat="rank_off_pass_yds" >32</td><td class="right " data-stat="rank_off_pass_td" >32</td><td class="right " data-stat="rank_off_pass_int" >27</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >31</td><td class="right " data-stat="rank_def_yds" >32</td><td class="right " data-stat="rank_def_pts" >30</td><td class="right " data-stat="rank_def_turnovers" >21</td><td class="right " data-stat="rank_def_rush_att" >23</td><td class="right " data-stat="rank_def_rush_yds" >18</td><td class="right " data-stat="rank_def_rush_td" >27</td><td class="right " data-stat="rank_def_rush_yds_per_att" >11</td><td class="right " data-stat="rank_def_fumbles_rec" >23</td><td class="right " data-stat="rank_def_pass_att" >30</td><td class="right " data-stat="rank_def_pass_yds" >32</td><td class="right " data-stat="rank_def_pass_td" >31</td><td class="right " data-stat="rank_def_pass_int" >14</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >32</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2006/">2006</a></th><td class="left " data-stat="team" ><a href="/teams/gnb/2006.htm" title="Green Bay Packers">GNB</a></td><td class="left " data-stat="coordinator_type" >HC</td><td class="right " data-stat="teams_in_league" >32</td><td class="right " data-stat="rank_win_percentage" >13</td><td class="right " data-stat="rank_takeaway_giveaway" >17</td><td class="right " data-stat="rank_points_diff" >23</td><td class="right " data-stat="rank_yds_diff" >13</td><td class="right " data-stat="rank_off_yds" >9</td><td class="right " data-stat="rank_off_pts" >22</td><td class="right " data-stat="rank_off_turnovers" >26</td><td class="right " data-stat="rank_off_rush_att" >21</td><td class="right " data-stat="rank_off_rush_yds" >23</td><td class="right " data-stat="rank_off_rush_td" >23</td><td class="right " data-stat="rank_off_rush_yds_per_att" >21</td><td class="right " data-stat="rank_off_fumbles_lost" >25</td><td class="right " data-stat="rank_off_pass_att" >1</td><td class="right " data-stat="rank_off_pass_yds" >8</td><td class="right " data-stat="rank_off_pass_td" >20</td><td class="right " data-stat="rank_off_pass_int" >19</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >17</td><td class="right " data-stat="rank_def_yds" >12</td><td class="right " data-stat="rank_def_pts" >25</td><td class="right " data-stat="rank_def_turnovers" >5</td><td class="right " data-stat="rank_def_rush_att" >10</td><td class="right " data-stat="rank_def_rush_yds" >13</td><td class="right " data-stat="rank_def_rush_td" >11</td><td class="right " data-stat="rank_def_rush_yds_per_att" >17</td><td class="right " data-stat="rank_def_fumbles_rec" >22</td><td class="right " data-stat="rank_def_pass_att" >17</td><td class="right " data-stat="rank_def_pass_yds" >17</td><td class="right " data-stat="rank_def_pass_td" >27</td><td class="right " data-stat="rank_def_pass_int" >3</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >14</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2007/">2007</a></th><td class="left " data-stat="team" ><a href="/teams/gnb/2007.htm" title="Green Bay Packers">GNB</a></td><td class="left " data-stat="coordinator_type" >HC</td><td class="right " data-stat="teams_in_league" >32</td><td class="right " data-stat="rank_win_percentage" >2</td><td class="right " data-stat="rank_takeaway_giveaway" >10</td><td class="right " data-stat="rank_points_diff" >3</td><td class="right " data-stat="rank_yds_diff" >5</td><td class="right " data-stat="rank_off_yds" >2</td><td class="right " data-stat="rank_off_pts" >4</td><td class="right " data-stat="rank_off_turnovers" >7</td><td class="right " data-stat="rank_off_rush_att" >28</td><td class="right " data-stat="rank_off_rush_yds" >21</td><td class="right " data-stat="rank_off_rush_td" >13</td><td class="right " data-stat="rank_off_rush_yds_per_att" >12</td><td class="right " data-stat="rank_off_fumbles_lost" >8</td><td class="right " data-stat="rank_off_pass_att" >6</td><td class="right " data-stat="rank_off_pass_yds" >2</td><td class="right " data-stat="rank_off_pass_td" >6</td><td class="right " data-stat="rank_off_pass_int" >11</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >3</td><td class="right " data-stat="rank_def_yds" >11</td><td class="right " data-stat="rank_def_pts" >6</td><td class="right " data-stat="rank_def_turnovers" >17</td><td class="right " data-stat="rank_def_rush_att" >15</td><td class="right " data-stat="rank_def_rush_yds" >14</td><td class="right " data-stat="rank_def_rush_td" >1</td><td class="right " data-stat="rank_def_rush_yds_per_att" >11</td><td class="right " data-stat="rank_def_fumbles_rec" >25</td><td class="right " data-stat="rank_def_pass_att" >13</td><td class="right " data-stat="rank_def_pass_yds" >12</td><td class="right " data-stat="rank_def_pass_td" >18</td><td class="right " data-stat="rank_def_pass_int" >6</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >12</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2008/">2008</a></th><td class="left " data-stat="team" ><a href="/teams/gnb/2008.htm" title="Green Bay Packers">GNB</a></td><td class="left " data-stat="coordinator_type" >HC</td><td class="right " data-stat="teams_in_league" >32</td><td class="right " data-stat="rank_win_percentage" >24</td><td class="right " data-stat="rank_takeaway_giveaway" >6</td><td class="right " data-stat="rank_points_diff" >14</td><td class="right " data-stat="rank_yds_diff" >17</td><td class="right " data-stat="rank_off_yds" >8</td><td class="right " data-stat="rank_off_pts" >5</td><td class="right " data-stat="rank_off_turnovers" >8</td><td class="right " data-stat="rank_off_rush_att" >14</td><td class="right " data-stat="rank_off_rush_yds" >17</td><td class="right " data-stat="rank_off_rush_td" >24</td><td class="right " data-stat="rank_off_rush_yds_per_att" >18</td><td class="right " data-stat="rank_off_fumbles_lost" >6</td><td class="right " data-stat="rank_off_pass_att" >9</td><td class="right " data-stat="rank_off_pass_yds" >8</td><td class="right " data-stat="rank_off_pass_td" >5</td><td class="right " data-stat="rank_off_pass_int" >12</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >10</td><td class="right " data-stat="rank_def_yds" >20</td><td class="right " data-stat="rank_def_pts" >22</td><td class="right " data-stat="rank_def_turnovers" >12</td><td class="right " data-stat="rank_def_rush_att" >24</td><td class="right " data-stat="rank_def_rush_yds" >26</td><td class="right " data-stat="rank_def_rush_td" >27</td><td class="right " data-stat="rank_def_rush_yds_per_att" >26</td><td class="right " data-stat="rank_def_fumbles_rec" >28</td><td class="right " data-stat="rank_def_pass_att" >17</td><td class="right " data-stat="rank_def_pass_yds" >12</td><td class="right " data-stat="rank_def_pass_td" >21</td><td class="right " data-stat="rank_def_pass_int" >3</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >12</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2009/">2009</a></th><td class="left " data-stat="team" ><a href="/teams/gnb/2009.htm" title="Green Bay Packers">GNB</a></td><td class="left " data-stat="coordinator_type" >HC</td><td class="right " data-stat="teams_in_league" >32</td><td class="right " data-stat="rank_win_percentage" >5</td><td class="right " data-stat="rank_takeaway_giveaway" >1</td><td class="right " data-stat="rank_points_diff" >2</td><td class="right " data-stat="rank_yds_diff" >1</td><td class="right " data-stat="rank_off_yds" >6</td><td class="right " data-stat="rank_off_pts" >3</td><td class="right " data-stat="rank_off_turnovers" >1</td><td class="right " data-stat="rank_off_rush_att" >15</td><td class="right " data-stat="rank_off_rush_yds" >14</td><td class="right " data-stat="rank_off_rush_td" >5</td><td class="right " data-stat="rank_off_rush_yds_per_att" >13</td><td class="right " data-stat="rank_off_fumbles_lost" >5</td><td class="right " data-stat="rank_off_pass_att" >10</td><td class="right " data-stat="rank_off_pass_yds" >7</td><td class="right " data-stat="rank_off_pass_td" >4</td><td class="right " data-stat="rank_off_pass_int" >2</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >10</td><td class="right " data-stat="rank_def_yds" >2</td><td class="right " data-stat="rank_def_pts" >7</td><td class="right " data-stat="rank_def_turnovers" >1</td><td class="right " data-stat="rank_def_rush_att" >3</td><td class="right " data-stat="rank_def_rush_yds" >1</td><td class="right " data-stat="rank_def_rush_td" >1</td><td class="right " data-stat="rank_def_rush_yds_per_att" >2</td><td class="right " data-stat="rank_def_fumbles_rec" >17</td><td class="right " data-stat="rank_def_pass_att" >20</td><td class="right " data-stat="rank_def_pass_yds" >5</td><td class="right " data-stat="rank_def_pass_td" >28</td><td class="right " data-stat="rank_def_pass_int" >1</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >6</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2010/">2010</a></th><td class="left " data-stat="team" ><a href="/teams/gnb/2010.htm" title="Green Bay Packers">GNB</a></td><td class="left " data-stat="coordinator_type" >HC</td><td class="right " data-stat="teams_in_league" >32</td><td class="right " data-stat="rank_win_percentage" >8</td><td class="right " data-stat="rank_takeaway_giveaway" >4</td><td class="right " data-stat="rank_points_diff" >2</td><td class="right " data-stat="rank_yds_diff" >7</td><td class="right " data-stat="rank_off_yds" >9</td><td class="right " data-stat="rank_off_pts" >10</td><td class="right " data-stat="rank_off_turnovers" >10</td><td class="right " data-stat="rank_off_rush_att" >20</td><td class="right " data-stat="rank_off_rush_yds" >24</td><td class="right " data-stat="rank_off_rush_td" >18</td><td class="right " data-stat="rank_off_rush_yds_per_att" >25</td><td class="right " data-stat="rank_off_fumbles_lost" >9</td><td class="right " data-stat="rank_off_pass_att" >16</td><td class="right " data-stat="rank_off_pass_yds" >5</td><td class="right " data-stat="rank_off_pass_td" >4</td><td class="right " data-stat="rank_off_pass_int" >9</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >3</td><td class="right " data-stat="rank_def_yds" >5</td><td class="right " data-stat="rank_def_pts" >2</td><td class="right " data-stat="rank_def_turnovers" >6</td><td class="right " data-stat="rank_def_rush_att" >6</td><td class="right " data-stat="rank_def_rush_yds" >18</td><td class="right " data-stat="rank_def_rush_td" >3</td><td class="right " data-stat="rank_def_rush_yds_per_att" >28</td><td class="right " data-stat="rank_def_fumbles_rec" >24</td><td class="right " data-stat="rank_def_pass_att" >13</td><td class="right " data-stat="rank_def_pass_yds" >5</td><td class="right " data-stat="rank_def_pass_td" >4</td><td class="right " data-stat="rank_def_pass_int" >2</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >3</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2011/">2011</a></th><td class="left " data-stat="team" ><a href="/teams/gnb/2011.htm" title="Green Bay Packers">GNB</a></td><td class="left " data-stat="coordinator_type" >HC</td><td class="right " data-stat="teams_in_league" >32</td><td class="right " data-stat="rank_win_percentage" >1</td><td class="right " data-stat="rank_takeaway_giveaway" >2</td><td class="right " data-stat="rank_points_diff" >2</td><td class="right " data-stat="rank_yds_diff" >17</td><td class="right " data-stat="rank_off_yds" >3</td><td class="right " data-stat="rank_off_pts" >1</td><td class="right " data-stat="rank_off_turnovers" >2</td><td class="right " data-stat="rank_off_rush_att" >26</td><td class="right " data-stat="rank_off_rush_yds" >27</td><td class="right " data-stat="rank_off_rush_td" >16</td><td class="right " data-stat="rank_off_rush_yds_per_att" >26</td><td class="right " data-stat="rank_off_fumbles_lost" >5</td><td class="right " data-stat="rank_off_pass_att" >14</td><td class="right " data-stat="rank_off_pass_yds" >3</td><td class="right " data-stat="rank_off_pass_td" >1</td><td class="right " data-stat="rank_off_pass_int" >2</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >1</td><td class="right " data-stat="rank_def_yds" >32</td><td class="right " data-stat="rank_def_pts" >19</td><td class="right " data-stat="rank_def_turnovers" >1</td><td class="right " data-stat="rank_def_rush_att" >5</td><td class="right " data-stat="rank_def_rush_yds" >14</td><td class="right " data-stat="rank_def_rush_td" >6</td><td class="right " data-stat="rank_def_rush_yds_per_att" >26</td><td class="right " data-stat="rank_def_fumbles_rec" >26</td><td class="right " data-stat="rank_def_pass_att" >32</td><td class="right " data-stat="rank_def_pass_yds" >32</td><td class="right " data-stat="rank_def_pass_td" >27</td><td class="right " data-stat="rank_def_pass_int" >1</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >30</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2012/">2012</a></th><td class="left " data-stat="team" ><a href="/teams/gnb/2012.htm" title="Green Bay Packers">GNB</a></td><td class="left " data-stat="coordinator_type" >HC</td><td class="right " data-stat="teams_in_league" >32</td><td class="right " data-stat="rank_win_percentage" >6</td><td class="right " data-stat="rank_takeaway_giveaway" >10</td><td class="right " data-stat="rank_points_diff" >7</td><td class="right " data-stat="rank_yds_diff" >9</td><td class="right " data-stat="rank_off_yds" >13</td><td class="right " data-stat="rank_off_pts" >5</td><td class="right " data-stat="rank_off_turnovers" >2</td><td class="right " data-stat="rank_off_rush_att" >16</td><td class="right " data-stat="rank_off_rush_yds" >20</td><td class="right " data-stat="rank_off_rush_td" >25</td><td class="right " data-stat="rank_off_rush_yds_per_att" >22</td><td class="right " data-stat="rank_off_fumbles_lost" >9</td><td class="right " data-stat="rank_off_pass_att" >16</td><td class="right " data-stat="rank_off_pass_yds" >9</td><td class="right " data-stat="rank_off_pass_td" >2</td><td class="right " data-stat="rank_off_pass_int" >1</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >12</td><td class="right " data-stat="rank_def_yds" >11</td><td class="right " data-stat="rank_def_pts" >11</td><td class="right " data-stat="rank_def_turnovers" >18</td><td class="right " data-stat="rank_def_rush_att" >13</td><td class="right " data-stat="rank_def_rush_yds" >17</td><td class="right " data-stat="rank_def_rush_td" >16</td><td class="right " data-stat="rank_def_rush_yds_per_att" >26</td><td class="right " data-stat="rank_def_fumbles_rec" >28</td><td class="right " data-stat="rank_def_pass_att" >22</td><td class="right " data-stat="rank_def_pass_yds" >11</td><td class="right " data-stat="rank_def_pass_td" >16</td><td class="right " data-stat="rank_def_pass_int" >8</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >7</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2013/">2013</a></th><td class="left " data-stat="team" ><a href="/teams/gnb/2013.htm" title="Green Bay Packers">GNB</a></td><td class="left " data-stat="coordinator_type" >HC</td><td class="right " data-stat="teams_in_league" >32</td><td class="right " data-stat="rank_win_percentage" >13</td><td class="right " data-stat="rank_takeaway_giveaway" >19</td><td class="right " data-stat="rank_points_diff" >16</td><td class="right " data-stat="rank_yds_diff" >8</td><td class="right " data-stat="rank_off_yds" >3</td><td class="right " data-stat="rank_off_pts" >8</td><td class="right " data-stat="rank_off_turnovers" >15</td><td class="right " data-stat="rank_off_rush_att" >12</td><td class="right " data-stat="rank_off_rush_yds" >7</td><td class="right " data-stat="rank_off_rush_td" >5</td><td class="right " data-stat="rank_off_rush_yds_per_att" >4</td><td class="right " data-stat="rank_off_fumbles_lost" >9</td><td class="right " data-stat="rank_off_pass_att" >18</td><td class="right " data-stat="rank_off_pass_yds" >6</td><td class="right " data-stat="rank_off_pass_td" >13</td><td class="right " data-stat="rank_off_pass_int" >17</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >7</td><td class="right " data-stat="rank_def_yds" >25</td><td class="right " data-stat="rank_def_pts" >24</td><td class="right " data-stat="rank_def_turnovers" >21</td><td class="right " data-stat="rank_def_rush_att" >12</td><td class="right " data-stat="rank_def_rush_yds" >25</td><td class="right " data-stat="rank_def_rush_td" >25</td><td class="right " data-stat="rank_def_rush_yds_per_att" >29</td><td class="right " data-stat="rank_def_fumbles_rec" >11</td><td class="right " data-stat="rank_def_pass_att" >9</td><td class="right " data-stat="rank_def_pass_yds" >24</td><td class="right " data-stat="rank_def_pass_td" >27</td><td class="right " data-stat="rank_def_pass_int" >26</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >25</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2014/">2014</a></th><td class="left " data-stat="team" ><a href="/teams/gnb/2014.htm" title="Green Bay Packers">GNB</a></td><td class="left " data-stat="coordinator_type" >HC</td><td class="right " data-stat="teams_in_league" >32</td><td class="right " data-stat="rank_win_percentage" >1</td><td class="right " data-stat="rank_takeaway_giveaway" >1</td><td class="right " data-stat="rank_points_diff" >3</td><td class="right " data-stat="rank_yds_diff" >6</td><td class="right " data-stat="rank_off_yds" >6</td><td class="right " data-stat="rank_off_pts" >1</td><td class="right " data-stat="rank_off_turnovers" >1</td><td class="right " data-stat="rank_off_rush_att" >14</td><td class="right " data-stat="rank_off_rush_yds" >11</td><td class="right " data-stat="rank_off_rush_td" >11</td><td class="right " data-stat="rank_off_rush_yds_per_att" >10</td><td class="right " data-stat="rank_off_fumbles_lost" >6</td><td class="right " data-stat="rank_off_pass_att" >20</td><td class="right " data-stat="rank_off_pass_yds" >8</td><td class="right " data-stat="rank_off_pass_td" >3</td><td class="right " data-stat="rank_off_pass_int" >1</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >1</td><td class="right " data-stat="rank_def_yds" >15</td><td class="right " data-stat="rank_def_pts" >13</td><td class="right " data-stat="rank_def_turnovers" >8</td><td class="right " data-stat="rank_def_rush_att" >23</td><td class="right " data-stat="rank_def_rush_yds" >23</td><td class="right " data-stat="rank_def_rush_td" >14</td><td class="right " data-stat="rank_def_rush_yds_per_att" >20</td><td class="right " data-stat="rank_def_fumbles_rec" >18</td><td class="right " data-stat="rank_def_pass_att" >22</td><td class="right " data-stat="rank_def_pass_yds" >10</td><td class="right " data-stat="rank_def_pass_td" >17</td><td class="right " data-stat="rank_def_pass_int" >7</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >8</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2015/">2015</a></th><td class="left " data-stat="team" ><a href="/teams/gnb/2015.htm" title="Green Bay Packers">GNB</a></td><td class="left " data-stat="coordinator_type" >HC</td><td class="right " data-stat="teams_in_league" >32</td><td class="right " data-stat="rank_win_percentage" >8</td><td class="right " data-stat="rank_takeaway_giveaway" >10</td><td class="right " data-stat="rank_points_diff" >11</td><td class="right " data-stat="rank_yds_diff" >19</td><td class="right " data-stat="rank_off_yds" >23</td><td class="right " data-stat="rank_off_pts" >15</td><td class="right " data-stat="rank_off_turnovers" >4</td><td class="right " data-stat="rank_off_rush_att" >12</td><td class="right " data-stat="rank_off_rush_yds" >12</td><td class="right " data-stat="rank_off_rush_td" >21</td><td class="right " data-stat="rank_off_rush_yds_per_att" >11</td><td class="right " data-stat="rank_off_fumbles_lost" >15</td><td class="right " data-stat="rank_off_pass_att" >18</td><td class="right " data-stat="rank_off_pass_yds" >25</td><td class="right " data-stat="rank_off_pass_td" >11</td><td class="right " data-stat="rank_off_pass_int" >3</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >31</td><td class="right " data-stat="rank_def_yds" >15</td><td class="right " data-stat="rank_def_pts" >12</td><td class="right " data-stat="rank_def_turnovers" >19</td><td class="right " data-stat="rank_def_rush_att" >18</td><td class="right " data-stat="rank_def_rush_yds" >21</td><td class="right " data-stat="rank_def_rush_td" >23</td><td class="right " data-stat="rank_def_rush_yds_per_att" >29</td><td class="right " data-stat="rank_def_fumbles_rec" >29</td><td class="right " data-stat="rank_def_pass_att" >13</td><td class="right " data-stat="rank_def_pass_yds" >6</td><td class="right " data-stat="rank_def_pass_td" >6</td><td class="right " data-stat="rank_def_pass_int" >9</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >11</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2016/">2016</a></th><td class="left " data-stat="team" ><a href="/teams/gnb/2016.htm" title="Green Bay Packers">GNB</a></td><td class="left " data-stat="coordinator_type" >HC</td><td class="right " data-stat="teams_in_league" >32</td><td class="right " data-stat="rank_win_percentage" >9</td><td class="right " data-stat="rank_takeaway_giveaway" >6</td><td class="right " data-stat="rank_points_diff" >8</td><td class="right " data-stat="rank_yds_diff" >16</td><td class="right " data-stat="rank_off_yds" >8</td><td class="right " data-stat="rank_off_pts" >4</td><td class="right " data-stat="rank_off_turnovers" >8</td><td class="right " data-stat="rank_off_rush_att" >29</td><td class="right " data-stat="rank_off_rush_yds" >20</td><td class="right " data-stat="rank_off_rush_td" >20</td><td class="right " data-stat="rank_off_rush_yds_per_att" >7</td><td class="right " data-stat="rank_off_fumbles_lost" >13</td><td class="right " data-stat="rank_off_pass_att" >5</td><td class="right " data-stat="rank_off_pass_yds" >7</td><td class="right " data-stat="rank_off_pass_td" >1</td><td class="right " data-stat="rank_off_pass_int" >7</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >17</td><td class="right " data-stat="rank_def_yds" >22</td><td class="right " data-stat="rank_def_pts" >21</td><td class="right " data-stat="rank_def_turnovers" >11</td><td class="right " data-stat="rank_def_rush_att" >6</td><td class="right " data-stat="rank_def_rush_yds" >8</td><td class="right " data-stat="rank_def_rush_td" >10</td><td class="right " data-stat="rank_def_rush_yds_per_att" >14</td><td class="right " data-stat="rank_def_fumbles_rec" >19</td><td class="right " data-stat="rank_def_pass_att" >16</td><td class="right " data-stat="rank_def_pass_yds" >31</td><td class="right " data-stat="rank_def_pass_td" >29</td><td class="right " data-stat="rank_def_pass_int" >4</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >30</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2017/">2017</a></th><td class="left " data-stat="team" ><a href="/teams/gnb/2017.htm" title="Green Bay Packers">GNB</a></td><td class="left " data-stat="coordinator_type" >HC</td><td class="right " data-stat="teams_in_league" >32</td><td class="right " data-stat="rank_win_percentage" >19</td><td class="right " data-stat="rank_takeaway_giveaway" >20</td><td class="right " data-stat="rank_points_diff" >23</td><td class="right " data-stat="rank_yds_diff" >27</td><td class="right " data-stat="rank_off_yds" >26</td><td class="right " data-stat="rank_off_pts" >21</td><td class="right " data-stat="rank_off_turnovers" >23</td><td class="right " data-stat="rank_off_rush_att" >27</td><td class="right " data-stat="rank_off_rush_yds" >17</td><td class="right " data-stat="rank_off_rush_td" >11</td><td class="right " data-stat="rank_off_rush_yds_per_att" >5</td><td class="right " data-stat="rank_off_fumbles_lost" >12</td><td class="right " data-stat="rank_off_pass_att" >14</td><td class="right " data-stat="rank_off_pass_yds" >25</td><td class="right " data-stat="rank_off_pass_td" >12</td><td class="right " data-stat="rank_off_pass_int" >28</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >31</td><td class="right " data-stat="rank_def_yds" >22</td><td class="right " data-stat="rank_def_pts" >26</td><td class="right " data-stat="rank_def_turnovers" >13</td><td class="right " data-stat="rank_def_rush_att" >24</td><td class="right " data-stat="rank_def_rush_yds" >17</td><td class="right " data-stat="rank_def_rush_td" >9</td><td class="right " data-stat="rank_def_rush_yds_per_att" >8</td><td class="right " data-stat="rank_def_fumbles_rec" >7</td><td class="right " data-stat="rank_def_pass_att" >5</td><td class="right " data-stat="rank_def_pass_yds" >23</td><td class="right " data-stat="rank_def_pass_td" >29</td><td class="right " data-stat="rank_def_pass_int" >20</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >29</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2018/">2018</a></th><td class="left " data-stat="team" ><a href="/teams/gnb/2018.htm" title="Green Bay Packers">GNB</a></td><td class="left " data-stat="coordinator_type" >HC</td><td class="right " data-stat="teams_in_league" >32</td><td class="right " data-stat="rank_win_percentage" >21</td><td class="right " data-stat="rank_takeaway_giveaway" >19</td><td class="right " data-stat="rank_points_diff" >19</td><td class="right " data-stat="rank_yds_diff" >14</td><td class="right " data-stat="rank_off_yds" >12</td><td class="right " data-stat="rank_off_pts" >14</td><td class="right " data-stat="rank_off_turnovers" >2</td><td class="right " data-stat="rank_off_rush_att" >32</td><td class="right " data-stat="rank_off_rush_yds" >22</td><td class="right " data-stat="rank_off_rush_td" >15</td><td class="right " data-stat="rank_off_rush_yds_per_att" >2</td><td class="right " data-stat="rank_off_fumbles_lost" >25</td><td class="right " data-stat="rank_off_pass_att" >3</td><td class="right " data-stat="rank_off_pass_yds" >9</td><td class="right " data-stat="rank_off_pass_td" >20</td><td class="right " data-stat="rank_off_pass_int" >1</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >21</td><td class="right " data-stat="rank_def_yds" >18</td><td class="right " data-stat="rank_def_pts" >22</td><td class="right " data-stat="rank_def_turnovers" >29</td><td class="right " data-stat="rank_def_rush_att" >27</td><td class="right " data-stat="rank_def_rush_yds" >22</td><td class="right " data-stat="rank_def_rush_td" >20</td><td class="right " data-stat="rank_def_rush_yds_per_att" >13</td><td class="right " data-stat="rank_def_fumbles_rec" >16</td><td class="right " data-stat="rank_def_pass_att" >8</td><td class="right " data-stat="rank_def_pass_yds" >12</td><td class="right " data-stat="rank_def_pass_td" >22</td><td class="right " data-stat="rank_def_pass_int" >29</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >21</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2020/">2020</a></th><td class="left " data-stat="team" ><a href="/teams/dal/2020.htm" title="Dallas Cowboys">DAL</a></td><td class="left " data-stat="coordinator_type" >HC</td><td class="right " data-stat="teams_in_league" >32</td><td class="right " data-stat="rank_win_percentage" >21</td><td class="right " data-stat="rank_takeaway_giveaway" >21</td><td class="right " data-stat="rank_points_diff" >25</td><td class="right " data-stat="rank_yds_diff" >21</td><td class="right " data-stat="rank_off_yds" >14</td><td class="right " data-stat="rank_off_pts" >17</td><td class="right " data-stat="rank_off_turnovers" >27</td><td class="right " data-stat="rank_off_rush_att" >15</td><td class="right " data-stat="rank_off_rush_yds" >17</td><td class="right " data-stat="rank_off_rush_td" >21</td><td class="right " data-stat="rank_off_rush_yds_per_att" >23</td><td class="right " data-stat="rank_off_fumbles_lost" >29</td><td class="right " data-stat="rank_off_pass_att" >2</td><td class="right " data-stat="rank_off_pass_yds" >8</td><td class="right " data-stat="rank_off_pass_td" >19</td><td class="right " data-stat="rank_off_pass_int" >17</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >22</td><td class="right " data-stat="rank_def_yds" >23</td><td class="right " data-stat="rank_def_pts" >28</td><td class="right " data-stat="rank_def_turnovers" >7</td><td class="right " data-stat="rank_def_rush_att" >31</td><td class="right " data-stat="rank_def_rush_yds" >31</td><td class="right " data-stat="rank_def_rush_td" >25</td><td class="right " data-stat="rank_def_rush_yds_per_att" >30</td><td class="right " data-stat="rank_def_fumbles_rec" >2</td><td class="right " data-stat="rank_def_pass_att" >2</td><td class="right " data-stat="rank_def_pass_yds" >11</td><td class="right " data-stat="rank_def_pass_td" >27</td><td class="right " data-stat="rank_def_pass_int" >23</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >21</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2021/">2021</a></th><td class="left " data-stat="team" ><a href="/teams/dal/2021.htm" title="Dallas Cowboys">DAL</a></td><td class="left " data-stat="coordinator_type" >HC</td><td class="right " data-stat="teams_in_league" >32</td><td class="right " data-stat="rank_win_percentage" >3</td><td class="right " data-stat="rank_takeaway_giveaway" >1</td><td class="right " data-stat="rank_points_diff" >2</td><td class="right " data-stat="rank_yds_diff" >4</td><td class="right " data-stat="rank_off_yds" >1</td><td class="right " data-stat="rank_off_pts" >1</td><td class="right " data-stat="rank_off_turnovers" >10</td><td class="right " data-stat="rank_off_rush_att" >12</td><td class="right " data-stat="rank_off_rush_yds" >9</td><td class="right " data-stat="rank_off_rush_td" >17</td><td class="right " data-stat="rank_off_rush_yds_per_att" >8</td><td class="right " data-stat="rank_off_fumbles_lost" >17</td><td class="right " data-stat="rank_off_pass_att" >6</td><td class="right " data-stat="rank_off_pass_yds" >2</td><td class="right " data-stat="rank_off_pass_td" >3</td><td class="right " data-stat="rank_off_pass_int" >7</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >4</td><td class="right " data-stat="rank_def_yds" >19</td><td class="right " data-stat="rank_def_pts" >7</td><td class="right " data-stat="rank_def_turnovers" >1</td><td class="right " data-stat="rank_def_rush_att" >11</td><td class="right " data-stat="rank_def_rush_yds" >16</td><td class="right " data-stat="rank_def_rush_td" >7</td><td class="right " data-stat="rank_def_rush_yds_per_att" >23</td><td class="right " data-stat="rank_def_fumbles_rec" >11</td><td class="right " data-stat="rank_def_pass_att" >25</td><td class="right " data-stat="rank_def_pass_yds" >20</td><td class="right " data-stat="rank_def_pass_td" >7</td><td class="right " data-stat="rank_def_pass_int" >1</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >18</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2022/">2022</a></th><td class="left " data-stat="team" ><a href="/teams/dal/2022.htm" title="Dallas Cowboys">DAL</a></td><td class="left " data-stat="coordinator_type" >HC</td><td class="right " data-stat="teams_in_league" >32</td><td class="right " data-stat="rank_win_percentage" >7</td><td class="right " data-stat="rank_takeaway_giveaway" >2</td><td class="right " data-stat="rank_points_diff" >5</td><td class="right " data-stat="rank_yds_diff" >8</td><td class="right " data-stat="rank_off_yds" >11</td><td class="right " data-stat="rank_off_pts" >4</td><td class="right " data-stat="rank_off_turnovers" >17</td><td class="right " data-stat="rank_off_rush_att" >6</td><td class="right " data-stat="rank_off_rush_yds" >9</td><td class="right " data-stat="rank_off_rush_td" >2</td><td class="right " data-stat="rank_off_rush_yds_per_att" >18</td><td class="right " data-stat="rank_off_fumbles_lost" >2</td><td class="right " data-stat="rank_off_pass_att" >19</td><td class="right " data-stat="rank_off_pass_yds" >14</td><td class="right " data-stat="rank_off_pass_td" >9</td><td class="right " data-stat="rank_off_pass_int" >30</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >9</td><td class="right " data-stat="rank_def_yds" >12</td><td class="right " data-stat="rank_def_pts" >5</td><td class="right " data-stat="rank_def_turnovers" >1</td><td class="right " data-stat="rank_def_rush_att" >27</td><td class="right " data-stat="rank_def_rush_yds" >22</td><td class="right " data-stat="rank_def_rush_td" >3</td><td class="right " data-stat="rank_def_rush_yds_per_att" >17</td><td class="right " data-stat="rank_def_fumbles_rec" >1</td><td class="right " data-stat="rank_def_pass_att" >11</td><td class="right " data-stat="rank_def_pass_yds" >8</td><td class="right " data-stat="rank_def_pass_td" >14</td><td class="right " data-stat="rank_def_pass_int" >7</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >8</td></tr>
# </tbody><tfoot><tr ><th scope="row" class="left " data-stat="year_id" >22 yrs</th><td class="left iz" data-stat="team" ></td><td class="left iz" data-stat="coordinator_type" ></td><td class="right iz" data-stat="teams_in_league" ></td><td class="right " data-stat="rank_win_percentage" >67</td><td class="right " data-stat="rank_takeaway_giveaway" >69</td><td class="right " data-stat="rank_points_diff" >63</td><td class="right " data-stat="rank_yds_diff" >58</td><td class="right " data-stat="rank_off_yds" >67</td><td class="right " data-stat="rank_off_pts" >72</td><td class="right " data-stat="rank_off_turnovers" >65</td><td class="right " data-stat="rank_off_rush_att" >45</td><td class="right " data-stat="rank_off_rush_yds" >51</td><td class="right " data-stat="rank_off_rush_td" >53</td><td class="right " data-stat="rank_off_rush_yds_per_att" >58</td><td class="right " data-stat="rank_off_fumbles_lost" >60</td><td class="right " data-stat="rank_off_pass_att" >64</td><td class="right " data-stat="rank_off_pass_yds" >69</td><td class="right " data-stat="rank_off_pass_td" >74</td><td class="right " data-stat="rank_off_pass_int" >67</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >60</td><td class="right " data-stat="rank_def_yds" >47</td><td class="right " data-stat="rank_def_pts" >48</td><td class="right " data-stat="rank_def_turnovers" >69</td><td class="right " data-stat="rank_def_rush_att" >51</td><td class="right " data-stat="rank_def_rush_yds" >45</td><td class="right " data-stat="rank_def_rush_td" >56</td><td class="right " data-stat="rank_def_rush_yds_per_att" >39</td><td class="right " data-stat="rank_def_fumbles_rec" >55</td><td class="right " data-stat="rank_def_pass_att" >54</td><td class="right " data-stat="rank_def_pass_yds" >52</td><td class="right " data-stat="rank_def_pass_td" >39</td><td class="right " data-stat="rank_def_pass_int" >68</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >47</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" >13 yrs</th><td class="left " data-stat="team" >GNB</td><td class="left " data-stat="coordinator_type" >HC</td><td class="right iz" data-stat="teams_in_league" ></td><td class="right " data-stat="rank_win_percentage" >72</td><td class="right " data-stat="rank_takeaway_giveaway" >73</td><td class="right " data-stat="rank_points_diff" >71</td><td class="right " data-stat="rank_yds_diff" >65</td><td class="right " data-stat="rank_off_yds" >72</td><td class="right " data-stat="rank_off_pts" >76</td><td class="right " data-stat="rank_off_turnovers" >77</td><td class="right " data-stat="rank_off_rush_att" >39</td><td class="right " data-stat="rank_off_rush_yds" >47</td><td class="right " data-stat="rank_off_rush_td" >53</td><td class="right " data-stat="rank_off_rush_yds_per_att" >61</td><td class="right " data-stat="rank_off_fumbles_lost" >68</td><td class="right " data-stat="rank_off_pass_att" >67</td><td class="right " data-stat="rank_off_pass_yds" >74</td><td class="right " data-stat="rank_off_pass_td" >79</td><td class="right " data-stat="rank_off_pass_int" >76</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >64</td><td class="right " data-stat="rank_def_yds" >53</td><td class="right " data-stat="rank_def_pts" >53</td><td class="right " data-stat="rank_def_turnovers" >64</td><td class="right " data-stat="rank_def_rush_att" >58</td><td class="right " data-stat="rank_def_rush_yds" >50</td><td class="right " data-stat="rank_def_rush_td" >63</td><td class="right " data-stat="rank_def_rush_yds_per_att" >43</td><td class="right " data-stat="rank_def_fumbles_rec" >38</td><td class="right " data-stat="rank_def_pass_att" >53</td><td class="right " data-stat="rank_def_pass_yds" >55</td><td class="right " data-stat="rank_def_pass_td" >38</td><td class="right " data-stat="rank_def_pass_int" >75</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >53</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" >5 yrs</th><td class="left " data-stat="team" >NOR</td><td class="left " data-stat="coordinator_type" >OC</td><td class="right iz" data-stat="teams_in_league" ></td><td class="right " data-stat="rank_win_percentage" >62</td><td class="right " data-stat="rank_takeaway_giveaway" >61</td><td class="right " data-stat="rank_points_diff" >48</td><td class="right " data-stat="rank_yds_diff" >44</td><td class="right " data-stat="rank_off_yds" >62</td><td class="right " data-stat="rank_off_pts" >69</td><td class="right " data-stat="rank_off_turnovers" >56</td><td class="right " data-stat="rank_off_rush_att" >48</td><td class="right " data-stat="rank_off_rush_yds" >52</td><td class="right " data-stat="rank_off_rush_td" >54</td><td class="right " data-stat="rank_off_rush_yds_per_att" >54</td><td class="right " data-stat="rank_off_fumbles_lost" >49</td><td class="right " data-stat="rank_off_pass_att" >61</td><td class="right " data-stat="rank_off_pass_yds" >65</td><td class="right " data-stat="rank_off_pass_td" >78</td><td class="right " data-stat="rank_off_pass_int" >65</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >58</td><td class="right " data-stat="rank_def_yds" >40</td><td class="right " data-stat="rank_def_pts" >37</td><td class="right " data-stat="rank_def_turnovers" >72</td><td class="right " data-stat="rank_def_rush_att" >48</td><td class="right " data-stat="rank_def_rush_yds" >40</td><td class="right " data-stat="rank_def_rush_td" >37</td><td class="right " data-stat="rank_def_rush_yds_per_att" >29</td><td class="right " data-stat="rank_def_fumbles_rec" >85</td><td class="right " data-stat="rank_def_pass_att" >57</td><td class="right " data-stat="rank_def_pass_yds" >47</td><td class="right " data-stat="rank_def_pass_td" >41</td><td class="right " data-stat="rank_def_pass_int" >53</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >37</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" >3 yrs</th><td class="left " data-stat="team" >DAL</td><td class="left " data-stat="coordinator_type" >HC</td><td class="right iz" data-stat="teams_in_league" ></td><td class="right " data-stat="rank_win_percentage" >71</td><td class="right " data-stat="rank_takeaway_giveaway" >78</td><td class="right " data-stat="rank_points_diff" >70</td><td class="right " data-stat="rank_yds_diff" >69</td><td class="right " data-stat="rank_off_yds" >76</td><td class="right " data-stat="rank_off_pts" >80</td><td class="right " data-stat="rank_off_turnovers" >47</td><td class="right " data-stat="rank_off_rush_att" >69</td><td class="right " data-stat="rank_off_rush_yds" >67</td><td class="right " data-stat="rank_off_rush_td" >61</td><td class="right " data-stat="rank_off_rush_yds_per_att" >52</td><td class="right " data-stat="rank_off_fumbles_lost" >53</td><td class="right " data-stat="rank_off_pass_att" >75</td><td class="right " data-stat="rank_off_pass_yds" >78</td><td class="right " data-stat="rank_off_pass_td" >71</td><td class="right " data-stat="rank_off_pass_int" >47</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >67</td><td class="right " data-stat="rank_def_yds" >47</td><td class="right " data-stat="rank_def_pts" >61</td><td class="right " data-stat="rank_def_turnovers" >94</td><td class="right " data-stat="rank_def_rush_att" >31</td><td class="right " data-stat="rank_def_rush_yds" >31</td><td class="right " data-stat="rank_def_rush_td" >67</td><td class="right " data-stat="rank_def_rush_yds_per_att" >30</td><td class="right " data-stat="rank_def_fumbles_rec" >89</td><td class="right " data-stat="rank_def_pass_att" >64</td><td class="right " data-stat="rank_def_pass_yds" >63</td><td class="right " data-stat="rank_def_pass_td" >53</td><td class="right " data-stat="rank_def_pass_int" >71</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >54</td></tr>
# <tr ><th scope="row" class="left " data-stat="year_id" >1 yr</th><td class="left " data-stat="team" >SFO</td><td class="left " data-stat="coordinator_type" >OC</td><td class="right iz" data-stat="teams_in_league" ></td><td class="right " data-stat="rank_win_percentage" >22</td><td class="right " data-stat="rank_takeaway_giveaway" >19</td><td class="right " data-stat="rank_points_diff" >3</td><td class="right " data-stat="rank_yds_diff" >3</td><td class="right " data-stat="rank_off_yds" >3</td><td class="right " data-stat="rank_off_pts" >9</td><td class="right " data-stat="rank_off_turnovers" >19</td><td class="right " data-stat="rank_off_rush_att" >41</td><td class="right " data-stat="rank_off_rush_yds" >50</td><td class="right " data-stat="rank_off_rush_td" >25</td><td class="right " data-stat="rank_off_rush_yds_per_att" >53</td><td class="right " data-stat="rank_off_fumbles_lost" >38</td><td class="right " data-stat="rank_off_pass_att" >6</td><td class="right " data-stat="rank_off_pass_yds" >3</td><td class="right " data-stat="rank_off_pass_td" >3</td><td class="right " data-stat="rank_off_pass_int" >19</td><td class="right " data-stat="rank_off_pass_net_yds_per_att" >6</td><td class="right " data-stat="rank_def_yds" >3</td><td class="right " data-stat="rank_def_pts" >9</td><td class="right " data-stat="rank_def_turnovers" >38</td><td class="right " data-stat="rank_def_rush_att" >31</td><td class="right " data-stat="rank_def_rush_yds" >47</td><td class="right " data-stat="rank_def_rush_td" >19</td><td class="right " data-stat="rank_def_rush_yds_per_att" >69</td><td class="right " data-stat="rank_def_fumbles_rec" >31</td><td class="right " data-stat="rank_def_pass_att" >9</td><td class="right " data-stat="rank_def_pass_yds" >3</td><td class="right " data-stat="rank_def_pass_td" >6</td><td class="right " data-stat="rank_def_pass_int" >59</td><td class="right " data-stat="rank_def_pass_net_yds_per_att" >3</td></tr>

#    </tfoot>

# </table>


# </div>
# -->


# </div>
